#Librerias

Son las librerias necesarias para la ejecución del codigo, con el cual define la arquitectura de una red neuronal del tipo convolucional o multicapa, para la identificación de los momentos de inversión (alcista o bajista) de un portafolio de inversión.

In [ ]:
#Ejecutar pip en la primera ejecución
#!pip install yahoo-fin --upgrade
#!pip install requests_html

from yahoo_fin import stock_info
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import random
import os, signal
import tensorflow as tf
from keras.layers import Dense,Dropout,Conv1D,Input,Flatten,MaxPooling1D
from keras.models import Sequential,load_model
from keras.optimizers import adam_v2
from sklearn.metrics import accuracy_score
%run '/content/drive/MyDrive/Escrito/Codigo/Portafolio/Datos.ipynb' import *
%run '/content/drive/MyDrive/Escrito/Codigo/Portafolio/Portafolio.ipynb' import *

     |████████████████████████████████| 81 kB 3.5 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 127 kB 18.4 MB/s 
     |████████████████████████████████| 103 kB 34.1 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=3fef69a8608f65183708abc0a979a9549194d1184b23666a983518b98ee86150
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=d7a11d7481aa03abae3f123fee26c90188fa953592cb438930bf10e2304dba12
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=d2a834fd6efd87581830a9e0f1a0fd07d2318f01e5007aae0e08cefa0fe15f31
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

#Variables de iniciales


Lista de las emisoras a extraer información, así como las fechas iniciales y finales de la información requerida

In [ ]:
fecha_inicio='2020-01-01'
fecha_final="2021-05-30"

#Carga empresas a evaluar
emp= pd.read_csv('/content/drive/MyDrive/Escrito/Codigo/Portafolio/Emisoras.csv',encoding='latin-1')   #Actualizar Ruta del archivo
emp.head()

#Definición de emisoras 
lista_emp= np.array(emp['Emisora'])

#Red Neuronal

Definición de la arquitectura de una red neuronal la cual puede ser Multicapa o Convolucional
 

In [ ]:
#Definición de las semillas del modelo
def set_seeds(seed=1000):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

#Creación de la Red Neuronal Multicapa
def RNN(entrada=10,capas=1, neuronas=[128], optimizer='Adam',dropout=False,td=0.1):
  red = Sequential()
  red.add(Dense(neuronas[0], input_dim=entrada,activation='relu'))
  for c in range(capas):
    if dropout:
      red.add(Dropout(td))
    red.add(Dense(neuronas[c], activation='relu'))
  red.add(Dense(1, activation='sigmoid'))
  red.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
  return red

#Creación de la Red Neuronal Convolucional
def CNN(entrada=10,capas=1, neuronas=[128], optimizer='Adam',dropout=False,td=0.1):
  red=Sequential()
  red.add(Conv1D(neuronas[0], activation='relu',kernel_size=1,input_shape=[entrada,1]))
  red.add(MaxPooling1D())
  for c in range(capas):
    if dropout:
      red.add(Dropout(td))
    red.add(Conv1D(neuronas[c], activation='relu',kernel_size=1))
    red.add(MaxPooling1D())
  red.add(Flatten())
  red.add(Dense(1, activation='sigmoid'))
  red.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'],)
  return red

#Implementación del modelo 

Funciones para la implementación de una red neuronal para la identificación de momentos del mercado y tomar la decisión de inversión

In [ ]:
#Implementación de red MLP para identificar el movimiento del portafolio en el mercado
def decision_MLP(path,emisoras,pesos,fecha_inicio,fecha_final):
  datos=datos1_port(path,emisoras,pesos,fecha_inicio,fecha_final)
  datos1=datos.copy()
  datos1=datos_ML1(datos1,3)
  x1=list(datos1.columns)
  print(datos1)
  y1=['clase']
  x1.remove('clase')
  x1.remove('clase2')
  x1.remove('open')
  x1.remove('low')
  x1.remove('high')
  x1.remove('volume')
  x1.sort()
  red=RNN(len(x1),1,[64],dropout=True,td=0.05)
  acc=-float('inf')
  for i in range(1000):
    modelo=red.fit(datos1[x1],datos1[y1],epochs=1)
    if modelo.history['accuracy'][0]>acc:
      red.save('decision.h5')
      acc=modelo.history['accuracy'][0]
      print ('Modelo Actualizado Guardado..')
  return datos1,x1,y1

#Implementación de red MLP para predecir el movimiento del mercado
def decision_CNN(path,emisoras,pesos,fecha_inicio,fecha_final):
  datos=datos1_port(path,emisoras,pesos,fecha_inicio,fecha_final)
  datos1=datos.copy()
  datos1=datos_ML1(datos1,3)
  x1=list(datos1.columns)
  y1=['clase']
  x1.remove('clase')
  x1.remove('clase2')
  x1.remove('open')
  x1.remove('low')
  x1.remove('high')
  x1.remove('volume')
  x1.sort()
  red=CNN(len(x1),1,[64],dropout=True,td=0.05)
  acc=-float('inf')
  for i in range(1000):    
    dat_conv= datos1[x1].to_numpy()
    dat_conv=dat_conv.reshape((datos1[x1].shape[0],datos1[x1].shape[1],1))
    modelo=red.fit(dat_conv,datos1[y1],epochs=1,verbose=0)
    if modelo.history['accuracy'][0]>acc:
      red.save('decision_cnn.h5')
      acc=modelo.history['accuracy'][0]
  return datos1,x1,y1


#Evaluación del Modelo

Función para la evaluación de la red neuronal implementada a partir de un conjunto de datos historicos.

In [ ]:
def eva_red(path,modelo,emisoras,pesos,fe_inicio,fe_fin):
  red=load_model(modelo)
  datos= datos1_eval(path,emisoras,pesos,fe_inicio,fe_fin+dt.timedelta(days=1))
  datos1=datos.copy()
  datos1=datos_ML1(datos1,3)
  n,m=datos1.shape
  y=['clase']
  x=list(datos1.columns)
  x.sort()
  dtest=datos1[-1:]
  dtest=datos1[-1:].to_numpy()
  dtest= dtest.reshape((1,m,1))
  predct=np.where(red.predict(dtest)>.5,1,0)
  return int(predct),datos['close'][datos.index[-1]]

#Evaluación de Red Neuronal Multicapa vs Red Neuronal Convolucional

Función de evaluación y comparación de Accuracies obtenidos de cada una de las redes neuronales, y definir la mejor evaluada.

In [ ]:
def comparativo(path,lista_emp,fecha_inicio,fecha_final,n=2,t=10):
  acc=[];acct=[]
  cacc=[];cacct=[]
  for _ in range(t):
    #n=2
    pesos=np.random.rand(n)
    pesos=pesos/sum(pesos)
    print(pesos)
    #pesos=[1/n]*n
    lista=np.random.choice(lista_emp,n)
    print(lista)
    datosm,xm,ym=decision_MLP(path,lista,pesos,fecha_inicio,fecha_final)
    datosc,xc,yc=decision_CNN(path,lista,pesos,fecha_inicio,fecha_final)
    red_final= load_model('decision.h5')
    cnn_final= load_model('decision_cnn.h5')
    pred=np.where(red_final.predict(datosm[xm])>.5,1,0)
    acc.append(np.sum(pred==datosm[ym])/datosm.shape[0])
    dtrain=datosc[xc].to_numpy()
    dtrain= dtrain.reshape((datosc[xc].shape[0],datosc[xc].shape[1],1))
    predc=np.where(cnn_final.predict(dtrain)>.5,1,0)
    cacc.append(np.sum(predc==datosc[yc])/datosc.shape[0])
    fecha_inicio_test='2021-03-01'
    fecha_final_test='2021-08-31'
    datos_test=datos1_port(path,lista,pesos,fecha_inicio_test,fecha_final_test)
    datos_test2=datos_test.copy()
    datos_test2=datos_ML1(datos_test2,3)
    predt=np.where(red_final.predict(datos_test2[xm])>.5,1,0)
    acct.append(np.sum(predt==datos_test2[ym])/datos_test2.shape[0])
    dtest=datos_test2[xc].to_numpy()
    dtest= dtest.reshape((datos_test2[xc].shape[0],datos_test2[xc].shape[1],1))
    predct=np.where(cnn_final.predict(dtest)>.5,1,0)
    cacct.append(np.sum(predct==datos_test2[yc])/datos_test2.shape[0])
    print('Red Multicapa')
    print('Training',sum(acc)/float(len(acc)),acc)  
    print('Test',sum(acct)/float(len(acct)),acct)  
    print('Red Convolucional')
    print('Training',sum(cacc)/float(len(cacc)),cacc)  
    print('Test',sum(cacct)/float(len(cacct)),cacct)  
  return acc,acct,cacc,cacct
    


[0.45977902 0.54022098]
['TERRA13.MX' 'WFC.MX']


#Evaluación de la Red Neuronal Convolucional

In [ ]:
def conv():
  acc=[];acct=[]
  for _ in range(10):
    n=5
    pesos=[1/n]*n
    lista=np.random.choice(lista_emp,n)
    print(lista)
    datos,x,y=decision_CNN(lista,pesos,fecha_inicio,fecha_final)
    red_final= load_model('decision.h5')
    pred=np.where(red_final.predict(datos[x])>.5,1,0)
    acc.append(np.sum(pred==datos[y])/datos.shape[0])
    fecha_inicio_test='2021-03-01'
    fecha_final_test='2021-08-31'
    datos_test=datos_port(lista,pesos,fecha_inicio_test,fecha_final_test)
    datos_test2=datos_test.copy()
    datos_test2=datos_ML1(datos_test2,3)
    predt=np.where(red_final.predict(datos_test2[x])>.5,1,0)
    acct.append(np.sum(predt==datos_test2[y])/datos_test2.shape[0])
    print('Training',sum(acc)/float(len(acc)))  
    print('Test',sum(acct)/float(len(acct)))  
